In [0]:
%sql
CREATE TABLE IF NOT EXISTS workspace.gold_serving.run_sst_result
USING DELTA
AS
SELECT
  r.time AS run_time,
  r.in_num,
  r.in_name,
  r.pk,

  s.number AS sst_number,
  s.in_num AS sst_in_num,
  s.in_name AS sst_in_name,
  s.sst_name,
  s.peak,
  s.eval_result,
  s.result AS sst_result,

  res.samp_id,
  res.as_id,
  res.Type,
  res.comp,
  res.unit,
  res.det_id,
  res.result AS assay_result,
  res.number_of_averaged_samples

FROM workspace.gold_serving.dimension_runs r

LEFT JOIN workspace.gold_serving.fact_sst s
  ON r.pk = s.fk

LEFT JOIN workspace.gold_serving.fact_results res
  ON r.pk = res.fk;


MERGE INTO workspace.gold_serving.run_sst_result AS target
USING (
  SELECT * FROM (
    SELECT
      r.time AS run_time,
      r.in_num,
      r.in_name,
      r.pk,

      s.number AS sst_number,
      s.in_num AS sst_in_num,
      s.in_name AS sst_in_name,
      s.sst_name,
      s.peak,
      s.eval_result,
      s.result AS sst_result,

      res.samp_id,
      res.as_id,
      res.Type,
      res.comp,
      res.unit,
      res.det_id,
      res.result AS assay_result,
      res.number_of_averaged_samples,

      ROW_NUMBER() OVER (
        PARTITION BY r.pk, s.number, res.samp_id
        ORDER BY r.time DESC
      ) AS rn
    FROM workspace.gold_serving.dimension_runs r
    LEFT JOIN workspace.gold_serving.fact_sst s ON r.pk = s.fk
    LEFT JOIN workspace.gold_serving.fact_results res ON r.pk = res.fk
  ) WHERE rn = 1
) AS source
ON target.pk = source.pk
AND target.run_time = source.run_time
AND target.samp_id = source.samp_id
AND target.sst_number = source.sst_number
WHEN MATCHED THEN
  UPDATE SET
    target.in_num = source.in_num,
    target.in_name = source.in_name,
    target.sst_in_num = source.sst_in_num,
    target.sst_in_name = source.sst_in_name,
    target.sst_name = source.sst_name,
    target.peak = source.peak,
    target.eval_result = source.eval_result,
    target.sst_result = source.sst_result,
    target.as_id = source.as_id,
    target.Type = source.Type,
    target.comp = source.comp,
    target.unit = source.unit,
    target.det_id = source.det_id,
    target.assay_result = source.assay_result,
    target.number_of_averaged_samples = source.number_of_averaged_samples
WHEN NOT MATCHED THEN
  INSERT (
    run_time,
    in_num,
    in_name,
    pk,
    sst_number,
    sst_in_num,
    sst_in_name,
    sst_name,
    peak,
    eval_result,
    sst_result,
    samp_id,
    as_id,
    Type,
    comp,
    unit,
    det_id,
    assay_result,
    number_of_averaged_samples
  )
  VALUES (
    source.run_time,
    source.in_num,
    source.in_name,
    source.pk,
    source.sst_number,
    source.sst_in_num,
    source.sst_in_name,
    source.sst_name,
    source.peak,
    source.eval_result,
    source.sst_result,
    source.samp_id,
    source.as_id,
    source.Type,
    source.comp,
    source.unit,
    source.det_id,
    source.assay_result,
    source.number_of_averaged_samples
  );

